## Topology 1

GARRETT PARZYCH

Much of this is taken from the FABRIC P4 tutorial

In [1]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

In [2]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Address,Location,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,RAM Available,RAM Capacity,RAM Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated
LOSA,"818 West 7th Street,Los Angeles, CA 90017-3407","(34.049043, -118.259476)",3,6,192,192,0,1536,1536,0,60600,60600,0,381,381,0,2,2,0,2,2,0,10,10,0,0,0,0,0,0,0
MASS,"100 Bigelow Street,Holyoke MA 01040","(42.202493000000004, -72.60787662257826)",3,6,192,192,0,1536,1536,0,60600,60600,0,381,381,0,2,2,0,2,2,0,10,10,0,2,2,0,3,3,0
WASH,"The Bexley, 1761 Old Meadow Road, McLean, VA 22102, United States of America","(38.91930235, -77.21183383681088)",3,6,152,192,40,1456,1536,80,59780,60600,820,369,381,12,0,2,2,2,2,0,10,10,0,2,2,0,3,3,0
FIU,"11001 SW 14th St,Miami, FL 33199","(25.754495891386522, -80.37232833001887)",5,10,270,320,50,2324,2560,236,115210,116400,1190,613,635,22,0,2,2,2,4,2,16,16,0,4,4,0,6,6,0
UCSD,"10100 Hopkins Dr, San Diego, CA 92121","(32.8881832, -117.2388161)",5,10,266,320,54,2136,2560,424,116030,116400,370,618,635,17,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
NEWY,"32 Sixth Avenue,New York, NY 10013","(43.453157655429585, -76.53364473809769)",2,4,128,128,0,1024,1024,0,9600,9600,0,254,254,0,3,3,0,3,3,0,8,8,0,0,0,0,0,0,0
CERN,"CERN - Meyrin site, Route de Meyrin, Prévessin, Prévessin-Moëns, Gex, Ain, Auvergne-Rhône-Alpes, Metropolitan France, 01280, France","(46.2338702, 6.0469869175479545)",6,12,102,384,282,1436,3072,1636,74110,75000,890,733,762,29,6,6,0,4,4,0,22,22,0,0,0,0,0,0,0
GPN,"5115 Oak Street, Kansas City, MO 64112","(39.03426274760282, -94.58260749540294)",5,10,320,320,0,2560,2560,0,116400,116400,0,635,635,0,2,2,0,4,4,0,16,16,0,4,4,0,6,6,0
CLEM,"340 Computer Court,Anderson, SC 29625","(34.586543500000005, -82.82128891709674)",3,6,100,192,92,1360,1536,176,60280,60600,320,375,381,6,2,2,0,0,2,2,10,10,0,2,2,0,3,3,0
GATECH,"760 West Peachtree Street NW Atlanta, GA 30308","(33.7753991, -84.3875488)",5,10,320,320,0,2560,2560,0,116400,116400,0,635,635,0,2,2,0,4,4,0,16,16,0,0,0,0,0,0,0


In [3]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

# Slice 
slice_name = 'project_fabric'

[site1,site2,site3] = fablib.get_random_sites(count=3)
print(f"Sites: {site1},{site2},{site3}")

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 100

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_subnet=IPv4Network('10.0.1.0/24')
h1_addr=IPv4Address('10.0.1.1')

h2_subnet=IPv4Network('10.0.2.0/24')
h2_addr=IPv4Address('10.0.2.2')

h3_subnet=IPv4Network('10.0.3.0/24')
h3_addr=IPv4Address('10.0.3.3')

host_cores = 2
host_ram = 8
host_disk = 10

net_h1_name = 'net_h1'
net_h2_name = 'net_h2'
net_h3_name = 'net_h3'

net_s1_s2_name = 'net_s1_s2'
net_s1_s3_name = 'net_s1_s3'

# All node properties
image = 'default_ubuntu_20'

Sites: CERN,MASS,TACC


In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site1,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1_iface_local = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()[0]
    s1_iface_to_s2 = s1.add_component(model='NIC_Basic', name="s1_switch_nic1").get_interfaces()[0]
    s1_iface_to_s3 = s1.add_component(model='NIC_Basic', name="s1_switch_nic2").get_interfaces()[0]

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site2,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2_iface_local = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()[0]
    s2_iface_to_s1 = s2.add_component(model='NIC_Basic', name="s2_switch_nic1").get_interfaces()[0]
    
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site3,  image=image, 
                        cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3_iface_local = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()[0]
    s3_iface_to_s1 = s3.add_component(model='NIC_Basic', name="s3_switch_nic1").get_interfaces()[0]
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site1, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h1_iface = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()[0]
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site2, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h2_iface = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()[0]
    
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site3, image=image,
                        cores=host_cores, ram=host_ram, disk=host_disk)
    h3_iface = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()[0]
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name=net_s1_s2_name, interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name=net_s1_s3_name, interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name=net_h1_name, interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name=net_h2_name, interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name=net_h3_name, interfaces=[s3_iface_local, h3_iface])
    
    #Submit Slice Request
    slice.submit() 
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()


Retry: 7, Time: 366 sec


ID,6f0db60f-3d4c-4253-a725-b050c7a4a48f
Name,project_fabric
Lease Expiration (UTC),2023-05-05 02:41:12 +0000
Lease Start (UTC),2023-05-04 02:41:13 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8994f849-0f21-478f-b578-b68d8669e948,s1,2,8,100,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe65:8b40,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe65:8b40,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f2f6d8c8-a79a-4bda-979a-2a2b4a7cdf81,s2,2,8,100,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.107,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.107,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ad0d438d-4f71-4a38-a2ec-6a862222830c,s3,2,8,100,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.75,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.75,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5e246f7e-dca0-4d15-aaa6-152d01825635,h1,2,8,10,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe79:a0d0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe79:a0d0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1ad841e5-25b5-4641-963b-7d3a4ee9bf67,h2,2,8,10,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.91,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.91,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
623757e3-3e73-459b-a77b-89ca3d23ad9f,h3,2,8,10,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.91,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.91,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Gateway,Subnet,State,Error
54744005-f343-4004-af76-9766ed288d2e,net_s1_s2,L2,L2STS,None,None,None,Active,
2b575e62-baf3-42c9-b4dd-f0ebdceb5893,net_s1_s3,L2,L2STS,None,None,None,Active,
d676a266-da97-4891-b7a6-a775109463cf,net_h1,L2,L2Bridge,CERN,None,None,Active,
b9a83715-7473-416f-8f49-9d062c2d5cbf,net_h2,L2,L2Bridge,MASS,None,None,Active,
73d0acd6-f6b6-4293-a7f1-1fca800fd157,net_h3,L2,L2Bridge,TACC,None,None,Active,



Time to stable 366 seconds
Running post_boot_config ... Time to post boot config 403 seconds


Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
s1-s1_switch_nic1-p1,s1,net_s1_s2,100,None,02:B5:53:89:2C:E6,ens7,ens7
s1-s1_switch_nic2-p1,s1,net_s1_s3,100,None,0A:2B:1C:A6:7C:94,ens9,ens9
s1-s1_local_nic-p1,s1,net_h1,100,None,06:CD:3C:FF:8A:7B,ens8,ens8
s2-s2_local_nic-p1,s2,net_h2,100,None,02:52:65:7C:CA:DE,ens8,ens8
s2-s2_switch_nic1-p1,s2,net_s1_s2,100,None,02:0D:69:D4:E4:A7,ens7,ens7
s3-s3_local_nic-p1,s3,net_h3,100,None,02:E9:67:A7:48:94,ens8,ens8
s3-s3_switch_nic1-p1,s3,net_s1_s3,100,None,02:6E:7D:FC:B0:1F,ens7,ens7
h1-h1_nic-p1,h1,net_h1,100,None,0A:82:F3:50:5F:D6,ens7,ens7
h2-h2_nic-p1,h2,net_h2,100,None,06:66:35:F6:23:F7,ens7,ens7
h3-h3_nic-p1,h3,net_h3,100,None,06:34:B9:B2:55:E2,ens7,ens7



Time to print interfaces 423 seconds


In [5]:
try:
    slice = fablib.get_slice(name=slice_name)

    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
8994f849-0f21-478f-b578-b68d8669e948,s1,2,8,100,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe65:8b40,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe65:8b40,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
f2f6d8c8-a79a-4bda-979a-2a2b4a7cdf81,s2,2,8,100,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.107,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.107,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ad0d438d-4f71-4a38-a2ec-6a862222830c,s3,2,8,100,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.75,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.75,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
5e246f7e-dca0-4d15-aaa6-152d01825635,h1,2,8,10,default_ubuntu_20,qcow2,cern-w1.fabric-testbed.net,CERN,ubuntu,2001:400:a100:3090:f816:3eff:fe79:a0d0,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe79:a0d0,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1ad841e5-25b5-4641-963b-7d3a4ee9bf67,h2,2,8,10,default_ubuntu_20,qcow2,mass-w2.fabric-testbed.net,MASS,ubuntu,205.172.170.91,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.91,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
623757e3-3e73-459b-a77b-89ca3d23ad9f,h3,2,8,10,default_ubuntu_20,qcow2,tacc-w3.fabric-testbed.net,TACC,ubuntu,129.114.110.91,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.91,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


In [6]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node.get_name()}: {node.get_ssh_command()}")
except Exception as e:
    print(f"Exception: {e}")

s1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe65:8b40
s2: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.107
s3: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.75
h1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3090:f816:3eff:fe79:a0d0
h2: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@205.172.170.91
h3: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@129.114.110.91


In [7]:
try:
    slice = fablib.get_slice(name=slice_name)
    
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

Name,Node,Network,Bandwidth,VLAN,MAC,Physical Device,Device
s1-s1_switch_nic1-p1,s1,net_s1_s2,100,None,02:B5:53:89:2C:E6,ens7,ens7
s1-s1_switch_nic2-p1,s1,net_s1_s3,100,None,0A:2B:1C:A6:7C:94,ens9,ens9
s1-s1_local_nic-p1,s1,net_h1,100,None,06:CD:3C:FF:8A:7B,ens8,ens8
s2-s2_local_nic-p1,s2,net_h2,100,None,02:52:65:7C:CA:DE,ens8,ens8
s2-s2_switch_nic1-p1,s2,net_s1_s2,100,None,02:0D:69:D4:E4:A7,ens7,ens7
s3-s3_switch_nic1-p1,s3,net_s1_s3,100,None,02:6E:7D:FC:B0:1F,ens7,ens7
s3-s3_local_nic-p1,s3,net_h3,100,None,02:E9:67:A7:48:94,ens8,ens8
h1-h1_nic-p1,h1,net_h1,100,None,0A:82:F3:50:5F:D6,ens7,ens7
h2-h2_nic-p1,h2,net_h2,100,None,06:66:35:F6:23:F7,ens7,ens7
h3-h3_nic-p1,h3,net_h3,100,None,06:34:B9:B2:55:E2,ens7,ens7


## Hosts

In [8]:
config_threads = {}

In [10]:
host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && sudo apt-get install net-tools && mkdir PROJECT" 

try:
    
    
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_os_iface.ip_link_up()
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread

    
    h2 = slice.get_node(name=h2_name)
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_os_iface.ip_link_up()
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    h3 = slice.get_node(name=h3_name)
    if type(ip_address(h3.get_management_ip())) is IPv6Address:
        h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_os_iface.ip_link_up()
    h3_config_thread = h3.execute_thread(host_config_script)
    config_threads[h3] = h3_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

 sudo: unable to resolve host 5e246f7e-dca0-4d15-aaa6-152d01825635-h1: Name or service not known
sudo: unable to resolve host 5e246f7e-dca0-4d15-aaa6-152d01825635-h1: Temporary failure in name resolution
sudo: unable to resolve host 5e246f7e-dca0-4d15-aaa6-152d01825635-h1: Name or service not known
sudo: unable to resolve host 5e246f7e-dca0-4d15-aaa6-152d01825635-h1: Name or service not known
 

In [18]:
h1.upload_file('files/send.py', 'PROJECT/send.py')
h1.upload_file('files/receive.py', 'PROJECT/receive.py')
h1.upload_file('files/server.py', 'PROJECT/server.py')
h1.upload_file('files/server_latency.py', 'PROJECT/server_latency.py')
h1.upload_file('files/client_latency.py', 'PROJECT/client_latency.py')
h1.upload_file('files/server_latency_basic.py', 'PROJECT/server_latency_basic.py')

h2.upload_file('files/send.py', 'PROJECT/send.py')
h2.upload_file('files/receive.py', 'PROJECT/receive.py')
h2.upload_file('files/server.py', 'PROJECT/server.py')
h2.upload_file('files/server_latency.py', 'PROJECT/server_latency.py')
h2.upload_file('files/client_latency.py', 'PROJECT/client_latency.py')
h2.upload_file('files/server_latency_basic.py', 'PROJECT/server_latency_basic.py')

h3.upload_file('files/send.py', 'PROJECT/send.py')
h3.upload_file('files/receive.py', 'PROJECT/receive.py')
h3.upload_file('files/server.py', 'PROJECT/server.py')
h3.upload_file('files/server_latency.py', 'PROJECT/server_latency.py')
h3.upload_file('files/client_latency.py', 'PROJECT/client_latency.py')
h3.upload_file('files/server_latency_basic.py', 'PROJECT/server_latency_basic.py')

<SFTPAttributes: [ size=1202 uid=1000 gid=1000 mode=0o100664 atime=1683172360 mtime=1683172360 ]>

In [42]:
h2.upload_file('files/server_latency_basic.py', 'PROJECT/server_latency_basic.py')
h3.upload_file('files/server_latency_basic.py', 'PROJECT/server_latency_basic.py')

<SFTPAttributes: [ size=2469 uid=1000 gid=1000 mode=0o100664 atime=1683193165 mtime=1683193165 ]>

## Switches

In [13]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
        
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
        
    s1.upload_file('files/project.p4', 'project.p4')
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()

In [14]:
try:
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)

    if type(ip_address(s2.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    s2.upload_file('files/project.p4', 'project.p4')
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command= f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    s2_config_thread = s2.execute_thread(command)
    config_threads[s2] = s2_config_thread

except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [15]:
try:
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)

    if type(ip_address(s3.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
     
    s3.upload_file('files/project.p4', 'project.p4')
    
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command = f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    s3_config_thread = s3.execute_thread(command)
    config_threads[s3] = s3_config_thread


except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [ ]:
try:
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc() 

In [71]:
slice.get_node("h3").download_file("/home/fabric/work/PROJECT/myproto_host1_client3_verylarge.txt", "/home/ubuntu/PROJECT/host1_verylarge.txt")

In [73]:
file = "/home/fabric/work/PROJECT/myproto_host1_client3_verylarge.txt"

with open(file) as data:
    sum = 0
    tot = 0
    for line in data:
        line2 = data.readline()
        
        line_s = line.split()
        line2_s = line2.split()
        
        t0 = float(line_s[0])
        t1 = float(line2_s[0])
        
        print(t1 - t0)
        sum += (t1 - t0)
        tot += 1
    print("avg = ", sum / tot)

0.1466660499572754
0.15061402320861816
0.1515820026397705
0.14641690254211426
0.14278602600097656
0.150299072265625
0.15047502517700195
0.14227795600891113
0.15027379989624023
0.1501150131225586
0.1506049633026123
0.15820908546447754
0.1452620029449463
0.14600586891174316
0.15022993087768555
0.15020108222961426
0.14226508140563965
0.14995694160461426
0.14595890045166016
0.15056085586547852
avg =  0.14853802919387818


## Delete

In [74]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()